# Crime Count Prediction Workflow
This Colab notebook walks through:
1. Data inspection (duplicates, missing values, info)
2. Visualization of missing data
3. Feature engineering & preparation
4. Prophet model training on **train.xlsx**
5. Monthly crime‑count predictions on **test (2).csv**
6. Model evaluation (MAE & RMSE)
7. Plotting actual vs. predicted counts

---

## 1  Install Libraries & Import Packages

In [ ]:
# Install Prophet and other utilities (quiet install)
!pip install prophet seaborn --quiet

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
from google.colab import files


## 2  Upload Datasets
Run the next cell and pick **train.xlsx** and **test (2).csv** from your computer.

In [ ]:
# Upload data files
uploaded = files.upload()

# Paths after upload
train_path = 'train.xlsx'
test_path = 'test (2).csv'

# Load datasets
train_df = pd.read_excel(train_path)
test_df  = pd.read_csv(test_path)

print(f"Train shape: {train_df.shape}")
print(f"Test  shape: {test_df.shape}")


## 3  Duplicate‑Row Counts

In [ ]:
print("Duplicate rows in TRAIN :", train_df.duplicated().sum())
print("Duplicate rows in TEST  :", test_df.duplicated().sum())


## 4  Missing‑Value Counts

In [ ]:
print("Missing values in TRAIN:\n", train_df.isna().sum())
print("\nMissing values in TEST:\n",  test_df.isna().sum())


## 5  Visualize Missingness

In [ ]:
plt.figure(figsize=(12,4))
sns.heatmap(train_df.isna(), cbar=False)
plt.title("TRAIN missing values")
plt.show()

plt.figure(figsize=(12,4))
sns.heatmap(test_df.isna(), cbar=False)
plt.title("TEST missing values")
plt.show()


## 6  Dataset Info & Summary Stats

In [ ]:
train_df.info()
print("\nColumns:", train_df.columns.tolist())
print("\nDescriptive statistics:\n", train_df.describe())

print("\nUnique values per column:")
for col in train_df.columns:
    print(f"{col}: {train_df[col].nunique()}")


## 7  Feature Engineering & Preparation
Adapt the column names below if your dataset uses different ones.

In [ ]:
# --- TRAIN dataset ---
# Assume columns: 'Date' (YYYY-MM-DD) and 'Incident_Counts'
train = train_df[['Date', 'Incident_Counts']].copy()
train.columns = ['ds', 'y']
train['ds'] = pd.to_datetime(train['ds'])

# --- TEST dataset ---
# If YEAR & MONTH exist, construct a date; else use 'Date'
if {'YEAR', 'MONTH'}.issubset(test_df.columns):
    test_df['ds'] = pd.to_datetime(test_df['YEAR'].astype(str) + '-' +
                                   test_df['MONTH'].astype(str).str.zfill(2) + '-01')
elif 'Date' in test_df.columns:
    test_df['ds'] = pd.to_datetime(test_df['Date'])

# Align target column name
if 'Incident_Counts' in test_df.columns:
    test_df.rename(columns={'Incident_Counts': 'y'}, inplace=True)


## 8  Model Training with Prophet

In [ ]:
m = Prophet()
m.fit(train)


## 9  Generate Monthly Predictions on Test Set

In [ ]:
future   = test_df[['ds']]  # Prophet expects a 'ds' column
forecast = m.predict(future)

pred_df = forecast[['ds', 'yhat']].copy()


## 10  Evaluate the Model

In [ ]:
# Merge actual y from test and predicted yhat
evaluation = test_df[['ds', 'y']].merge(pred_df, on='ds', how='left')

mae  = mean_absolute_error(evaluation['y'], evaluation['yhat'])
rmse = mean_squared_error(evaluation['y'], evaluation['yhat'], squared=False)

print(f"MAE : {mae:.3f}")
print(f"RMSE: {rmse:.3f}")


## 11  Plot Actual vs. Predicted

In [ ]:
plt.figure(figsize=(14,6))
plt.plot(evaluation['ds'], evaluation['y'],    label='Actual')
plt.plot(evaluation['ds'], evaluation['yhat'], label='Predicted')
plt.title("Monthly Crime Counts: Actual vs. Prophet Prediction")
plt.xlabel("Date")
plt.ylabel("Incidents")
plt.legend()
plt.show()


## 12  Download Predictions

In [ ]:
evaluation.to_csv('crime_predictions.csv', index=False)
files.download('crime_predictions.csv')
